In [1]:
import pandas as pd

In [2]:
movies = pd.read_json("movies.json")
reviews = pd.read_json("reviews.json")
reviews.head()

,user_id,item_id,rating,timestamp
0,0,50,5,1997-12-04 15:55:49
1,0,172,5,1997-12-04 15:55:49
2,0,133,1,1997-12-04 15:55:49
3,196,242,3,1997-12-04 15:55:49
4,186,302,3,1998-04-04 19:22:22


In [3]:
reviews.drop(['timestamp'], inplace=True, axis=1)

In [4]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99710 entries, 0 to 99709
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   user_id  99710 non-null  int64
 1   item_id  99710 non-null  int64
 2   rating   99710 non-null  int64
dtypes: int64(3)
memory usage: 2.3 MB


In [5]:
mostReviewed = 3
topN = reviews.groupby('item_id').size()

In [6]:
topN = topN.sort_values(ascending=False).reset_index()[:mostReviewed]
topN

,item_id,0
0,50,583
1,258,507
2,100,506


In [7]:
top3DF = reviews[reviews['item_id'].isin(topN['item_id'])]

In [8]:
pivotDF = top3DF.pivot(index='user_id', columns='item_id', values='rating')

In [9]:
pivotDF = pivotDF.dropna(axis=0, how="all")

In [10]:
pivotDF = pivotDF.fillna(-1)

In [11]:
pivotDF.head()

item_id,50,100,258
user_id,,,
0,5.0,-1.0,-1.0
1,5.0,5.0,5.0
2,5.0,5.0,3.0
3,-1.0,-1.0,2.0
4,5.0,-1.0,5.0


In [13]:
from sklearn.cluster import MiniBatchKMeans
>>> import numpy as np

In [14]:
#use 1-10% of data
kb = MiniBatchKMeans(n_clusters=2,random_state=0,batch_size=6)#not used:,n_init="auto"

In [15]:
kb.fit(pivotDF)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1043: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5120 or by setting the environment variable OMP_NUM_THREADS=1
  warnings.warn(


MiniBatchKMeans(batch_size=6, n_clusters=2, random_state=0)

In [17]:
#centroids coordinates
kb.cluster_centers_

array([[3.88235294, 2.61764706, 3.82352941],
       [1.47368421, 2.02631579, 0.57894737]])

In [18]:
from sklearn.metrics import silhouette_score

In [20]:
ss = silhouette_score(pivotDF, kb.labels_)

In [21]:
ss

0.28421773323138216

In [25]:
#use diff one for spotify dataset
scores = {}
for i in range(2, 10):
    ms = MiniBatchKMeans(n_clusters=i,random_state=0,batch_size=6)
    ms = kb.fit(pivotDF)
    if(len(set(kb.labels_)) > 1):
        ss = silhouette_score(pivotDF, kb.labels_)
        scores[i] = ss

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1043: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5120 or by setting the environment variable OMP_NUM_THREADS=1
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1043: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5120 or by setting the environment variable OMP_NUM_THREADS=1
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1043: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5120 or by setting the environment variable OMP_NUM_THREADS=1
  warnings.warn(
C:\Prog

In [26]:
scores

{2: 0.28421773323138216,
 3: 0.28421773323138216,
 4: 0.28421773323138216,
 5: 0.28421773323138216,
 6: 0.28421773323138216,
 7: 0.28421773323138216,
 8: 0.28421773323138216,
 9: 0.28421773323138216}

In [34]:
kb = MiniBatchKMeans(n_clusters=8,random_state=0,batch_size=6)#not used:,n_init="auto"
kb.fit(pivotDF)
kb.cluster_centers_
ss = silhouette_score(pivotDF, kb.labels_)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1043: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5120 or by setting the environment variable OMP_NUM_THREADS=1
  warnings.warn(


In [35]:
ss

0.6205231475924143

In [37]:
pivotDF['Cluster'] = kb.labels_

In [38]:
pivotDF.head()

item_id,50,100,258,Cluster
user_id,,,,
0,5.0,-1.0,-1.0,7
1,5.0,5.0,5.0,6
2,5.0,5.0,3.0,6
3,-1.0,-1.0,2.0,5
4,5.0,-1.0,5.0,3


In [44]:
user8 = reviews[reviews['user_id'] == 8]

In [45]:
user8

,user_id,item_id,rating
93,8,338,4
518,8,550,3
684,8,22,5
1089,8,50,5
3840,8,182,5
4641,8,79,4
6026,8,294,3
6117,8,457,1
6122,8,385,1
7033,8,89,4


In [46]:
user4 = reviews[reviews['user_id'] == 4]

In [47]:
user4

,user_id,item_id,rating
1249,4,264,3
1328,4,303,5
2201,4,361,5
2523,4,357,4
3272,4,260,4
5952,4,356,3
12135,4,294,5
13875,4,288,4
16284,4,50,5
18908,4,354,5


In [ ]:
#user 8 has not seen 303 which is highly rated